In [1]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload your dataset


Saving updatedTraining.csv to updatedTraining.csv


In [2]:
import pandas as pd
data = pd.read_csv('updatedTraining.csv')  # Replace with your file name


In [3]:
print(data.head())
print(data.info())


        dur proto service state  spkts  dpkts  sbytes  dbytes attack_cat  \
0  0.000011   udp       -   INT      2      0     496       0     Normal   
1  0.000008   udp       -   INT      2      0    1762       0     Normal   
2  0.000005   udp       -   INT      2      0    1068       0     Normal   
3  0.000006   udp       -   INT      2      0     900       0     Normal   
4  0.000010   udp       -   INT      2      0    2126       0     Normal   

   label  
0      0  
1      0  
2      0  
3      0  
4      0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dur         82332 non-null  float64
 1   proto       82332 non-null  object 
 2   service     82332 non-null  object 
 3   state       82332 non-null  object 
 4   spkts       82332 non-null  int64  
 5   dpkts       82332 non-null  int64  
 6   sbytes      82332 non-null  int64  

In [4]:
!pip install transformers


In [5]:
!pip install torch scikit-learn pandas


In [13]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example tokenization
def tokenize_features(row):
    return tokenizer(
        f"{row['proto']} {row['service']} {row['state']} {row['sbytes']} {row['dbytes']}",
        max_length=128,
        padding='max_length',
        truncation=True,
        return_tensors="pt"
    )

tokenized_data = data.apply(tokenize_features, axis=1)


In [14]:
labels = data['label'].values


In [15]:
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    tokenized_data, labels, test_size=0.2, random_state=42
)


In [16]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [11]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(
    torch.tensor(train_texts), torch.tensor(train_labels)
)
test_dataset = TensorDataset(
    torch.tensor(test_texts), torch.tensor(test_labels)
)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)


ValueError: could not determine the shape of object type 'BatchEncoding'

In [19]:
# Combine relevant columns into a single string for each row
text_data = data.apply(
    lambda row: f"{row['proto']} {row['service']} {row['state']} {row['sbytes']} {row['dbytes']}",
    axis=1
).tolist()  # Convert to list of strings

# Tokenize the combined text data
tokenized = tokenizer(
    text_data,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Extract tokenized components
input_ids = tokenized['input_ids']
attention_mask = tokenized['attention_mask']
labels = torch.tensor(data['label'].values)  # Ensure labels are in tensor format


In [20]:
from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)


In [21]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)


In [22]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)


In [23]:
from transformers import BertForSequenceClassification

# Load BERT model with classification head
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [25]:
from transformers import AdamW

# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define loss function (automatically handled during training in Hugging Face models)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
from torch.nn import functional as F

# Training parameters
epochs = 3

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        # Move batch to GPU
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Total Loss: {total_loss}")


Epoch 1
Total Loss: 1001.3308443953283
Epoch 2
Total Loss: 1763.8569887941703
Epoch 3
Total Loss: 2388.1222058162093


In [27]:
from sklearn.metrics import classification_report

# Put model in evaluation mode
model.eval()

# Store predictions and true labels
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        # Convert logits to class predictions
        preds = torch.argmax(F.softmax(logits, dim=1), axis=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

# Print classification report
print(classification_report(true_labels, predictions, target_names=["Normal", "Anomalous"]))


              precision    recall  f1-score   support

      Normal       0.80      0.99      0.88      7418
   Anomalous       0.99      0.80      0.88      9049

    accuracy                           0.88     16467
   macro avg       0.89      0.89      0.88     16467
weighted avg       0.90      0.88      0.88     16467



In [28]:
# Save model
model.save_pretrained('./bert_anomaly_detector')

# Save tokenizer
tokenizer.save_pretrained('./bert_anomaly_detector')


('./bert_anomaly_detector/tokenizer_config.json',
 './bert_anomaly_detector/special_tokens_map.json',
 './bert_anomaly_detector/vocab.txt',
 './bert_anomaly_detector/added_tokens.json')

In [29]:
from google.colab import files
!zip -r bert_anomaly_detector.zip ./bert_anomaly_detector
files.download('bert_anomaly_detector.zip')


  adding: bert_anomaly_detector/ (stored 0%)
  adding: bert_anomaly_detector/vocab.txt (deflated 53%)
  adding: bert_anomaly_detector/tokenizer_config.json (deflated 75%)
  adding: bert_anomaly_detector/special_tokens_map.json (deflated 42%)
  adding: bert_anomaly_detector/model.safetensors (deflated 7%)
  adding: bert_anomaly_detector/config.json (deflated 49%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>